In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error,mean_squared_error

from tbats import BATS, TBATS

from green_city.utils import span
from green_city.plotting import plot_decomposition
plt.rcParams['figure.figsize'] = [25, 5]

In [ ]:
df = (pd.read_csv("../data/preprocessed/Building_5.csv")
      .astype({'datetime': 'datetime64'}))
df = df.assign(tday = df.index.map(lambda x: x//24))
      #.set_index('datetime'))
len(df)

# Time series prediction for equipment power usage
- [?] which timewindow do I want to predict
- absolute power consumption for next hour/ next 6h/ next day

#### strategy
- compute trends and seasonalities ones such that they don't have to be re-computed when fitting to new data
- we may more want to go for 'recall' for peaks/overestimation because the potential loss when not having enough energy in store may overweight other costs

### Metric and Baseline
- if I only want to predict one hour, of sth. aggregated for a day, I can just use pre-day.

In [ ]:
#1. daily sum
df_daily =  df.groupby('tday').sum()[["equipment_electric_power_kWh"]].rename(columns={"equipment_electric_power_kWh": "actual"})
df_daily.actual.plot();

In [ ]:
plot_acf(df_daily.actual, lags=370);

1. eliminate yearly trend

In [ ]:
def add_seasonal_w_y_cols(df, to_row):
    df["trend"] = np.nan
    df["seasonal_year"] = np.nan
    df["seasonal_week"] = np.nan
    
    df_given = df.loc[:to_row]
    df_decomp = seasonal_decompose(df_given.actual, model='add', period=365, extrapolate_trend=1)
    df_given["trend"] = df_decomp.trend
    df_given["seasonal_year"] = df_decomp.seasonal
    df_decomp2 = seasonal_decompose(df_given.actual - df_given.seasonal_year, model='add', period=7)
    df_given["seasonal_week"] = df_decomp2.seasonal


In [ ]:
split_pos = int(len(df_daily)*(1 - 1/4))
add_seasonal_w_y_cols(df_daily, split_pos)

X = df_daily[:split_pos]
Y = df_daily[split_pos:]

#plot_acf(df_daily.actual[182:-182] - df_daily.trend[182:-182] - df_daily.seasonal[182:-182], lags=130)
plot_acf(X.actual - X.seasonal_year, lags=130)
plot_acf(X.actual - X.seasonal_year - X.seasonal_week, lags=130);

In [ ]:
len(df_daily)-split_pos
df_daily.seasonal_year[split_pos:] = df_daily.seasonal_year[split_pos-2*365:-2*365]
df_daily.seasonal_week[split_pos:] = df_daily.seasonal_week[split_pos-700:-700]
df_daily.trend[split_pos:] = df_daily.trend[split_pos]

df_daily["pred_by_decomp"] = df_daily.trend + df_daily.seasonal_year + df_daily.seasonal_week

In [ ]:
df_daily.actual.plot(color="green")
df_daily.trend.plot()
df_daily.pred_by_decomp.plot(color="red")
df_daily[:split_pos].actual.plot(color="blue")

In [ ]:
df_daily.seasonal_year.plot()
df_daily.seasonal_week.plot()

In [ ]:
df_daily.loc[split_pos-50:split_pos+100].pred_by_decomp.plot(color="red")
df_daily.loc[split_pos-50:split_pos+100].actual.plot(color="green")
df_daily.loc[split_pos-50:split_pos].actual.plot(color="black")
#next learn sth on the prediction error

### Predicting the residual

In [ ]:
df_daily["pred_error"] = df_daily["pred_by_decomp"] - df_daily["actual"]
plot_acf(df_daily["pred_error"], lags=70);

#error_estimator = TBATS()

from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(df_daily["pred_error"], order=(1,0,1))
model_fit = model.fit()
y_pred = model_fit.predict(dynamic=False)
plt.plot(y_pred)
y_pred
#fitted_error_model = error_estimator.fit(df_daily.iloc[:split_pos]["pred_error"])
#forecasted_error = fitted_error_model.forecast(steps=140)

In [ ]:
df_daily["pred_by_decomp_corrected"] = df_daily["pred_by_decomp"]
df_daily["pred_by_decomp_corrected"][split_pos:split_pos+140] += forecasted_error

In [ ]:
df_daily[split_pos-5:split_pos+5][["actual", "pred_by_decomp", "pred_by_decomp_corrected"]].plot()

In [ ]:

from pmdarima.arima import auto_arima
# fit auto-ARIMA
#also do stationarity test!!!
auto_model = auto_arima(df_daily[:split_pos]["pred_error"], start_p=0, start_q=0,
                         test='adf',
                         max_p=3, max_q=3,
                         start_P=0, seasonal=False,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

auto_model.summary()


In [ ]:
yy = auto_model.predict()
len(yy)

In [ ]:
df_daily["pred_by_decomp_corrected"][split_pos:split_pos+10] += yy

In [ ]:

df_daily[split_pos-4:split_pos+50][["actual", "pred_by_decomp", "pred_by_decomp_corrected"]].plot()

In [ ]:
df_daily[["pred_error"]].boxplot()
print(mean_absolute_error(df_daily[split_pos:]["actual"],df_daily[split_pos:]["pred_by_decomp"]))
print(mean_squared_error(df_daily[split_pos:]["actual"],df_daily[split_pos:]["pred_by_decomp"]))


## smoothed decomp

In [ ]:
daily_decomp = seasonal_decompose(df_daily.actual[:-len(df_daily)//4].rolling(21).mean()[21:], model='add', period=365)
plot_decomposition(daily_decomp)


In [ ]:
#predict seasonal

estimator1 = TBATS()
fitted_model1 = estimator1.fit(daily_decomp.trend[200:800])
forcasted_trend = fitted_model1.forecast(steps=14)
#plt.plot(forcasted_trend)

# Throwing the data at predictors

In [ ]:
df_daily_2 =  df.groupby('tday').sum()[["equipment_electric_power_kWh"]].rename(columns={"equipment_electric_power_kWh": "actual"})

In [ ]:
# Create estimator
estimator2 = TBATS(seasonal_periods=[7, 365])

# Fit model
fitted_model2 = estimator2.fit(df_daily_2.actual[:1000])

In [ ]:
# Forecast 14 steps ahead
y_forecasted = fitted_model2.forecast(steps=14)
forecast = pd.Series(np.nan, index = df_daily_2.index)
forecast[1000:1014] = y_forecasted
df_daily_2['forecast_14d'] = forecast

# Forecast 140 steps ahead
y_forecasted = fitted_model2.forecast(steps=140)
forecast = pd.Series(np.nan, index = df_daily_2.index)
forecast[1000:1140] = y_forecasted
df_daily_2['forecast_140d'] = forecast

In [ ]:
print(fitted_model2.summary())

In [ ]:
#fit with less data
# Create estimator
estimator3 = TBATS(seasonal_periods=[7])

# Fit model
fitted_model3 = estimator3.fit(df_daily_2.actual[600:1000])

In [ ]:
fitted_updated_model3 = estimator3.fit(df_daily_2.actual[600:1010])

In [ ]:
# Forecast 4 steps ahead
y_forecasted = fitted_model3.forecast(steps=14)
forecast3 = pd.Series(np.nan, index = df_daily_2.index)
forecast3[1000:1014] = y_forecasted
df_daily_2['forecast_4d'] = forecast3

In [ ]:
#print(fitted_model.summary())

In [ ]:
plt.plot(df_daily_2[350:1140].actual, color="green")
plt.plot(df_daily_2[350:1000].actual, color="black")
plt.plot(forecast, color="red")
plt.plot(df_daily_2.forecast_140d, color="orange")

In [ ]:
plt.plot(df_daily_2[920:1060]);

In [ ]:
print(mean_absolute_error(df_daily_2[1000:1140]['actual'], df_daily_2[1000:1140]['forecast_140d']))
print(mean_squared_error(df_daily_2[1000:1140]['actual'], df_daily_2[1000:1140]['forecast_140d']))
pd.DataFrame(df_daily_2[1000:1140]['actual'] - df_daily_2[1000:1140]['forecast_140d']).boxplot()

## Look into weekly trends

In [ ]:
#some of these consecetive weeks look very similar
len(df_daily_2.actual)
total_weeks = 1460//7 - 1
print(total_weeks)
#for i in range(151,158):
for i in range(152,156):
    plt.plot(df_daily_2[i*7:(i+1)*7].to_numpy())

In [ ]:
df_daily_2

## Manual seasonal decomposition

In [ ]:
power_series = df.set_index('datetime').equipment_electric_power_kWh
fig, (ax1, ax2) = plt.subplots(1, 2)
power_series.plot(ax=ax1)
power_series.loc[span('2008-08-04', '2008-08-12')].plot(ax=ax2);


In [ ]:
# Autocorrelation
plot_acf(power_series, lags=30); #use I(d) for d>0?

In [ ]:
#smooth daily and observe weekly trends
daily_power = df.groupby('tday').mean().equipment_electric_power_kWh
fig, (ax1, ax2) = plt.subplots(1, 2)
daily_power.plot(ax=ax1);
plot_acf(daily_power, lags=100, ax=ax2); #use I(d) for d>0?

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
df.equipment_electric_power_kWh.rolling(24).mean().plot(ax=ax1)
df.equipment_electric_power_kWh.rolling(24*7*4).mean().plot(ax=ax2); #[?] can I just choose some arbitrary rolling such that my season and trend look nice?

In [ ]:
# need to first factor out trends and seasonalities
rolling_window = 24*7*2
rolling_window = 24*7
smooth_4w_power = df.equipment_electric_power_kWh.rolling(rolling_window).mean()
decomp = seasonal_decompose(smooth_4w_power.loc[24*7*4:], model='add', period=365*24)
plot_decomposition(decomp)

In [ ]:
res = decomp.resid
res.plot()
decomp_2 = seasonal_decompose(res.rolling(24).mean().loc[6000:30000], model='add', period=7*24)
plot_decomposition(decomp_2)
fig, (ax) = plt.subplots(1, 1)
decomp_2.seasonal.loc[10000:10000+24*7*3].plot(ax=ax)

# Testing quality of prediction simulation

In [ ]:
temp = "outdoor_temp"
hum = "outdoor_hum"
diffuse_solar = "diffuse_solar_W_m2"
direct_solar = "direct_solar_W_m2"
pred_features = [temp, hum, diffuse_solar, direct_solar]

###################################
# change here which of the four features you want to investigate
feature = pred_features[0]
###################################

display(df.columns)

In [ ]:
#sns.pairplot(difs)
pdf = pd.concat([
    df[feature],
    df[f"pred_6h_{feature}"].shift(6),
    df[f"pred_12h_{feature}"].shift(12),
    df[f"pred_24h_{feature}"].shift(24),
], axis=1).iloc[24:, :]

In [ ]:
pred_temp_cols = [
    f'pred_6h_{feature}',
    f'pred_12h_{feature}',
    f'pred_24h_{feature}'
]

X = pdf[[f"pred_12h_{feature}", f"pred_24h_{feature}"]]
y = pdf[feature]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
pdf_test = pdf.loc[X_test.index, :].assign(
    pred_12h_better = lr.predict(X_test),
    pred_error = lambda x: x[f"pred_6h_{feature}"] - x[feature],
    pred_error_12 = lambda x: x[f"pred_12h_{feature}"] - x[feature],
    pred_error_24 = lambda x: x[f"pred_24h_{feature}"] - x[feature],
    pred_better_error = lambda x: x["pred_12h_better"] - x[feature],
)

In [ ]:
t = 3354
pdf.iloc[t:t+50, :].plot()

In [ ]:
pdf_test[["pred_error", "pred_better_error"]].rename(columns={
    "pred_error": "error of 6 hour prediction from csv",
    "pred_better_error": "error of my 12 hour prediction",
    }).boxplot()

In [ ]:
pdf_test[["pred_error", "pred_error_12", "pred_error_24"]].hist(bins=98)

In [ ]:
print(f"   csv 24h prediction corr with actual {feature}:", pdf_test[f"pred_24h_{feature}"].corr(pdf[feature]))
print(f"   csv 12h prediction corr with actual {feature}:", pdf_test[f"pred_12h_{feature}"].corr(pdf[feature]))
print(f"   csv  6h prediction corr with actual {feature}:", pdf_test[f"pred_6h_{feature}"].corr(pdf[feature]))
print(f"fitted 12h prediction corr with actual {feature}:", pdf_test["pred_12h_better"].corr(pdf[feature]))